<a href="https://colab.research.google.com/github/yvessong23/ML-TEAM-15/blob/main/BinBuddiesCNNFlatten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import TensorFlow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns

In [ ]:
%pip install numpy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hashlib, os
from PIL import Image
%pip install kagglehub
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zlatan599/garbage-dataset-classification")
rgb_path = "cleaned/garbage-dataset-rgb"
print("Path to dataset files:", path)

hashes = {}
duplicates = []

def file_hash(filePath):
    with open(filePath, "rb") as f: # Open and read files for hashing
        file_bytes = f.read()
        return hashlib.md5(file_bytes).hexdigest()

for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        h = file_hash(file_path)

        if h in hashes:
            print(f"Duplicate found! Removing {file_path} (same as {hashes[h]})")
            try:
                os.remove(file_path)
                duplicates.append((file_path, hashes[h]))
            except OSError as e:
                print(f"Warning: Could not remove {file_path} due to read-only filesystem. Error: {e}")
        else:
            hashes[h] = file_path
            try:
                # Preserve the dataset’s subfolder structure
                rel_dir = os.path.relpath(root, path)
                save_dir = os.path.join(rgb_path, rel_dir)
                os.makedirs(save_dir, exist_ok=True)

                save_path = os.path.join(save_dir, file)

                # Skip non-image files (like metadata.csv)
                if not file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                    continue

                with Image.open(file_path) as img:
                    img.convert("RGB").save(save_path)

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

rgb_path = "cleaned/garbage-dataset-rgb"
base_path = os.path.join(rgb_path, "Garbage_Dataset_Classification", "images")
DATASET_DIR = base_path

# Set up class names
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
num_classes = len(class_names)

# Ensure 'duplicates' is defined. It is expected to be populated by cell WJM9MyG1JvjH.
# If cell WJM9MyG1JvjH hasn't been run, 'duplicates' will be an empty list, and thus
# no files will be filtered as duplicates in this cell.
if 'duplicates' not in globals():
    duplicates = []

# Verify and collect all image paths and their labels
image_paths = []
labels = []

duplicate_filepaths = {f[0] for f in duplicates} # Extract only the file paths from the duplicates list

for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join(base_path, class_name)
    if os.path.exists(class_dir):
        class_images = [f for f in os.listdir(class_dir)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        print(f"{class_name}: {len(class_images)} images")

        for img_file in class_images:
            full_path = os.path.join(class_dir, img_file)
            if full_path not in duplicate_filepaths: # Only add if not a duplicate
                image_paths.append(full_path)
                labels.append(class_idx)
    else:
        print(f"Warning: {class_dir} does not exist")

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
import numpy as np

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SIZE   = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load all validation images into memory, then split into val + test
val_generator_full = val_test_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

val_data = []
val_labels = []

for i in range(len(val_generator_full)):
    x, y = val_generator_full[i]
    val_data.append(x[0])
    val_labels.append(y[0])

val_data = np.array(val_data)
val_labels = np.array(val_labels)

X_val, X_test, y_val, y_test = train_test_split(
    val_data,
    val_labels,
    test_size=0.5,
    random_state=42,
    stratify=np.argmax(val_labels, axis=1)
)

val_generator_data = (X_val, y_val)
test_generator_data = (X_test, y_test)

In [ ]:
model_flatten = models.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # Block 1
    # Block 1
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPool2D(2),

    # Block 2
    layers.Conv2D(128, (5,5), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPool2D(2),

    # Block 3
    layers.Conv2D(256, (3,3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.MaxPool2D(2),

    # Block 4
   layers.Conv2D(512, (3,3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.MaxPool2D(2),

    # Flatten Head
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dense(6, activation='softmax'),
])

In [ ]:
print(tf.config.list_physical_devices("GPU"))

checkpoint = ModelCheckpoint(
    'best_model_flatten.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)
model_flatten.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_flatten.summary()

history = model_flatten.fit(
    train_generator,
    validation_data=val_generator_data,
    epochs=30,
    callbacks=[checkpoint]
)

test_loss, test_acc = model_flatten.evaluate(X_test,
    y_test,
    batch_size=BATCH_SIZE,
    verbose=2
)

In [ ]:
test_images, test_labels = test_generator_data

y_probs = model_flatten.predict(test_images)

y_pred = np.argmax(y_probs, axis=1)
y_true = np.argmax(test_labels, axis=1)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')